In [1]:
import numpy as np
import pickle
import sys
import itk
from itkTypes import itkCType
import os

In [4]:
# NRRD Files
directory = '/home/will.dolan/Desktop/Jupyter/CTAandCTP/NRRD-Files/CTAT-006/'

# Saved NRRD Files
directory2 = '/home/will.dolan/Desktop/Jupyter/CTAandCTP/NRRD-Files/CTAT-006-Reg2/'

# Mask Name and Location
MASK_DIRECTORY = '/home/will.dolan/Desktop/Jupyter/CTAandCTP/NRRD-Files/CTAT-006/mask2.mha'

# ITKTubeTK binary location
tubetk = '/home/will.dolan/src/ITKTubeTK-Release/bin/RegisterImages'

pic_folder = directory
pic_folder = os.listdir(pic_folder)

Dimension = 3
PixelType = itk.ctype('float')
ImageType = itk.Image[PixelType, Dimension]

out = ImageType.New()

start = itk.Index[Dimension]()
start[0] = 0  
start[1] = 0  
start[2] = 0  

pic_folder.sort()
img0 = itk.imread(directory + pic_folder[0])

size = itk.Size[Dimension]()
size[0] = itk.size(img0)[0]
size[1] = itk.size(img0)[1]
size[2] = itk.size(img0)[2]


num_images = len(pic_folder)

imgdatasize = np.zeros(4, dtype=int)
imgdatasize[0] = num_images
imgdatasize[1] = size[2]
imgdatasize[2] = size[1]
imgdatasize[3] = size[0]
imgdata = np.zeros(imgdatasize)
imgdata.shape
        
for image in range(num_images):
    if image != 0:
        oscmd = (tubetk + ' ' + directory + pic_folder[0] + ' ' + directory + pic_folder[image] + ' --resampledImage ' + directory2 + pic_folder[image] + ' --rigidMaxIterations 3000 --registration Rigid --metric MeanSqrd --fixedImageMask ' + MASK_DIRECTORY + ' --sampleFromOverlap')
        os.system(oscmd)
    else:
        oscmd = ('cp ' + directory + pic_folder[image] + ' ' + directory2 + pic_folder[image])
        os.system(oscmd)
    img = itk.imread(directory2 + pic_folder[image])
    imgdata[image,:,:,:] = itk.GetArrayFromImage(img)


    print(pic_folder[image])

10-VPCT-DynMulti4D-1.5-H20f-8.nrrd


KeyboardInterrupt: 

In [ ]:
outdata = imgdata.max(0).astype('h')

out = itk.GetImageFromArray(outdata)
out.CopyInformation(img0)
final = itk.imwrite(out, '/home/will.dolan/Desktop/Jupyter/CTAandCTP/Composites/CTAT-006_Max(mask2).nrrd')

    